# Importing Libraries

In [41]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [42]:
cur = os.getcwd()

In [43]:
df = pd.read_csv(cur+"\\data\\data.zip")
df = df.drop('Unnamed: 0', 1)

C:\Users\Anubhav\AppData\Local\Temp/ipykernel_17276/1479601220.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('Unnamed: 0', 1)


In [44]:
df

,trade Id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch
0,1274557543,43160.00,0.00056,24.169600,1646092800000,True,True
1,1274557544,43160.01,0.00052,22.443205,1646092800003,False,True
2,1274557545,43160.01,0.00182,78.551218,1646092800004,False,True
3,1274557546,43160.01,0.00631,272.339663,1646092800005,False,True
4,1274557547,43160.00,0.00229,98.836400,1646092800006,True,True
...,...,...,...,...,...,...,...
2871280,1277428823,44736.04,0.08083,3616.014113,1646233096536,False,True
2871281,1277428824,44736.04,0.08864,3965.402586,1646233096536,False,True
2871282,1277428825,44737.45,0.00535,239.345358,1646233096556,True,True
2871283,1277428826,44737.45,0.00167,74.711541,1646233096567,True,True


# Basic Logic of Market Making System

In [45]:
"""
value = (bid+ask) /2
spread = (ask-bid)+profit_target
skew = -position*risk_adjustment


buy_quote=value+skew - spread
sell_quote = value + skew + spread


In this problem,

bid = value*99.97/100
ask = value*100.03/100

spread = value*0.06/100 + value*0.03/100
spread = 0.09*value/100

skew = -0*1=0
skew = -1
skew = -2
skew = -3
skew = -4 (5th Position)

In order 

buy_quote (nth position) = 99.91*value/100 + skew(n_position)
sell_quote (nth position) = 100.09*value/100 + skew(n_position)

"""



'\nvalue = (bid+ask) /2\nspread = (ask-bid)+profit_target\nskew = -position*risk_adjustment\n\n\nbuy_quote=value+skew - spread\nsell_quote = value + skew + spread\n\n\nIn this problem,\n\nbid = value*99.97/100\nask = value*100.03/100\n\nspread = value*0.06/100 + value*0.03/100\nspread = 0.09*value/100\n\nskew = -0*1=0\nskew = -1\nskew = -2\nskew = -3\nskew = -4 (5th Position)\n\nIn order \n\nbuy_quote (nth position) = 99.91*value/100 + skew(n_position)\nsell_quote (nth position) = 100.09*value/100 + skew(n_position)\n\n'

In [46]:
df['bid_quote']=df['price']*99.97/100
df['ask_quote']=df['price']*100.03/100

In [47]:
df.drop(df[df['quoteQty']<1000].index,inplace=True)

# Final DataFrame File

In [48]:
df

,trade Id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch,bid_quote,ask_quote
8,1274557551,43160.01,0.02358,1017.713036,1646092800010,False,True,43147.061997,43172.958003
9,1274557552,43160.01,0.02329,1005.196633,1646092800010,False,True,43147.061997,43172.958003
12,1274557555,43160.00,0.04072,1757.475200,1646092800045,True,True,43147.052000,43172.948000
22,1274557565,43156.43,0.02362,1019.354877,1646092800045,True,True,43143.483071,43169.376929
24,1274557567,43156.15,0.02321,1001.654242,1646092800045,True,True,43143.203155,43169.096845
...,...,...,...,...,...,...,...,...,...
2871273,1277428816,44734.74,0.03000,1342.042200,1646233096446,False,True,44721.319578,44748.160422
2871274,1277428817,44734.74,0.02864,1281.202954,1646233096450,False,True,44721.319578,44748.160422
2871280,1277428823,44736.04,0.08083,3616.014113,1646233096536,False,True,44722.619188,44749.460812
2871281,1277428824,44736.04,0.08864,3965.402586,1646233096536,False,True,44722.619188,44749.460812


In [49]:
buy_orders =[43147.061997]
sell_orders=[43172.958003]

In [50]:
def logic():

    Net_Position=0 #For Calculation of Hedging Purposes
    buy_orders =[43147.061997] # Bid_Order_Book
    sell_orders=[43172.958003] # Ask_Order_Book

    buying_trades=[] #Prices at which Btc was bought @ 1000USD 
    selling_trades=[] #Prices at which Btc was sold @ 1000USD 
    
    total_trades=0 #Total Trades Done
    for i in range(1,len(df['price'])):

        buy_orders.sort()
        sell_orders.sort()
        

        if (df['price'].iloc[i]<= buy_orders[-1] and Net_Position<5):  #If the current price goes lower than Bid_Order_Book
            total_trades= total_trades+1
            
            Net_Position+=1
            buying_trades.append(buy_orders[-1]) #Adding the Buying Trades Positions
            buy_orders.pop(-1)
            
            if (len(sell_orders)<5):
                
                buy_orders.append(df['bid_quote'].iloc[i])    #Modifying the bids and asks
                sell_orders.append(df['ask_quote'].iloc[i])
                
            
            else:
                buy_orders.append(df['bid_quote'].iloc[i])     #When 4000 usd trades are taken place on the side of buying

                                                                #It starts Buying the farthest bids
                sell_orders.pop(-1)
                

                
                sell_orders.append(df['ask_quote'].iloc[i])

            #print('Bid_Size =',buy_orders)
            #print('Ask_Size =',sell_orders)
            #print("LTP =",df['price'].iloc[i])
            #print(".......................")
            #print(i)

            

        if (df['price'].iloc[i]>= sell_orders[0] and Net_Position>-5):
            total_trades= total_trades+1
            selling_trades.append(sell_orders[0])
            Net_Position-=1
            sell_orders.pop(0)
            if (len(buy_orders)<5):
                
                buy_orders.append(df['bid_quote'].iloc[i])
                sell_orders.append(df['ask_quote'].iloc[i])
            
            
            else:    
                sell_orders.append(df['ask_quote'].iloc[i])
                
                
                buy_orders.pop(0)
                

                buy_orders.append(df['bid_quote'].iloc[i])


            #print('Bid_Size =',buy_orders)
            #print('Ask_Size =',sell_orders)
            #print("LTP =",df['price'].iloc[i])
            #print(".......................")
            #print(i)

            

        

            
        


        
        
            

        
          
                

    print(i)
    return total_trades,buying_trades,selling_trades

In [51]:
Number_of_Trades, buying_trades, selling_trades=logic()

819840


In [52]:
print ("Total Number of Trades =",Number_of_Trades)
print ("Money Invested =", Number_of_Trades*1000)

Total Number of Trades = 1815
Money Invested = 1815000


In [53]:
len(selling_trades)-len(buying_trades)

5

In [54]:
data = {'Buying':buying_trades,
        'Selling': selling_trades[5:]}


In [55]:
result = pd.DataFrame(data,columns=['Buying','Selling'])

In [56]:
result['diff']=result['Selling']-result['Buying']

In [57]:
sum(result['diff'])

1866.9047710010273

In [58]:
result['Money']=0
result['Money'].iloc[1]=result['diff'].iloc[1]+result['diff'].iloc[0]
for i in range(2,len(result)):
    result['Money'].iloc[i]=result['diff'].iloc[i]+result['Money'].iloc[i-1]
result

C:\Users\Anubhav\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Buying,Selling,diff,Money
0,43187.020006,43184.341417,-2.678589,0.000000
1,43184.890645,43196.795152,11.904507,9.225918
2,43173.923936,43197.655410,23.731474,32.957392
3,43171.394695,43197.655410,26.260715,59.218107
4,43185.260534,43193.064033,7.803499,67.021606
...,...,...,...,...
900,43456.849033,43483.041000,26.191967,1735.310388
901,43463.577014,43490.343190,26.766176,1762.076564
902,43463.577014,43492.163736,28.586722,1790.663286
903,43457.518832,43492.483832,34.965000,1825.628286


In [59]:
print ("Maximum Drawdown=", min(result['Money']))

Maximum Drawdown= -2814.0232139992877


In [60]:
print ("PNL =",sum(result['diff'])*100/(Number_of_Trades*1000))

PNL = 0.10285976699730177


In [61]:
wins_range=0
win_count=0
for i in range(len(result)):
    
    if (result['diff'].iloc[i]>0):
        wins_range+=result['diff'].iloc[i]
        win_count+=1

print ("Average Win=",wins_range/win_count)

Average Win= 16.214895861853662


In [62]:
loss_range=0
loss_count=0
for i in range(len(result)):
    
    if (result['diff'].iloc[i]<0):
        loss_range+=result['diff'].iloc[i]
        loss_count+=1

print ("Average_Loss=",loss_range/loss_count)

Average_Loss= -31.574566764924466
